# QA indicator records:

This notebook contains a QA of the queries implemented for the impact calculation in the main LG application. The main purpose is to update these calculations according to the improvements on the methodology: https://docs.google.com/document/d/1IDuYWOllQ2fTf2ZeBUmOtEZpqht2rqMot3G9W3CjEjE/edit#

As summary the formulas to compute each of the landgriffon impact indicators are shown below:

    Probability purchase area (ppa) = (ha / total ha) * Purchase Volume (tonnes)

### Water impacts 

        water imapct (m3/yr) = (BWF * 0.001 / Prod all crops) * ppa
    
    equal to:
    
        water imapct (m3/yr) = (BWF * 0.001 / Prod all crops) * (ha / total ha) * Volume
     
    therefore, water risk would be:
    
        water risk (m3/t*yr) = (BWF * 0.001 / Prod all crops) 
    

### Land impact:

        Land impact (ha/yr)  = (Harvested area (ha) / Production) *  ppa
        
     equal to:
   
        Land impact (ha/yr)  = (Harvested area (ha) / Production) *  (Ha / total ha) * Volume
        
     therefore, the land risk would be:
     
         Land risk (ha/t*yr) = (Harvested area (ha) / Production)
    
### Deforestation:

        Deforestation impact (ha/yr) = land impact (ha) * deforestation mask (unitless)
    
    equal to:
    
        Deforestation impact (ha/yr) = (Harvested area (ha) / Production) *  ppa * deforestation mask (unitless)
        Deforestation impact (ha/yr) = (Harvested area (ha) / Production) *  (Ha / total ha) * Volume * deforestation mask (unitless)
    
    therefore, the deforestation risk would be:
    
        Deforestation risk (ha/t*yr)= (Harvested area (ha) / Production)* deforestation mask (unitless)
        
### Carbon:

        Carbon impact (tCO2e/yr) = net forest carbo emisions (t CO2e) * Deforestation impact (ha)
    
    equal to: 
    
        Carbon impact (tCO2e/yr) = net forest carbo emisions (t CO2e) * (Harvested area (ha) / Production) *  ppa * deforestation mask (unitless)
        Carbon impact (tCO2e/yr) = net forest carbo emisions (t CO2e) * (Harvested area (ha) / Production) *  (Ha / total ha) * Volume * deforestation mask (unitless)
        
    therefore, the carbon risk would be:
    
         Carbon risk (tCO2e/t*yr) = net forest carbo emisions (t CO2e) * (Harvested area (ha) / Production)  * deforestation mask (unitless)
     
    
### Biodiversity:

        Biodiversity  impact  (PDF/yr)  = PSL(PDF m⁻²) * 10⁴(m² ha⁻¹) * Deforestation impact (ha)
    
    equal to: 
    
        Biodiversity  impact  (PDF/yr)  = PSL(PDF m⁻²) * 10⁴(m² ha⁻¹) * (Harvested area (ha) / Production) *  ppa * deforestation mask (unitless)
        Biodiversity  impact  (PDF/yr)  = PSL(PDF m⁻²) * 10⁴(m² ha⁻¹) * (Harvested area (ha) / Production) *   (Ha / total ha) * Volume * deforestation mask (unitless)
        
    therefore, the biodiversity risk would be:
    
         Biodiversity  risk  (PDF/t*yr)  = PSL(PDF m⁻²) * 10⁴(m² ha⁻¹) * (Harvested area (ha) / Production) * deforestation mask (unitless)
         
         

## Summary :

As the coputation of all these indicators can not be very performant if we considet the ppa approach, we can simplify it by computing the average of the risk in my region and multiply that value by the purchased volume. This will result on the following formulas:

    water impact (m3/yr) = avg((BWF * 0.001 / Prod all crops)) * Volume
    
    Land impact (ha/yr) = avg((Harvested area (ha) / Production)) * Volume
    
    Deforestation impact (ha/yr)= avg((Harvested area (ha) / Production)* deforestation mask (unitless)) * Volume
    
    Carbon impact (tCO2e/yr) = avg(net forest carbo emisions (t CO2e) * (Harvested area (ha) / Production)  * deforestation mask (unitless)) * Volume
    
    Biodiversity  impact  (PDF/t*yr)  = avg(PSL(PDF m⁻²) * 10⁴(m² ha⁻¹) * (Harvested area (ha) / Production) * deforestation mask (unitless)) * Volume

In [1]:
# import libraries
from psycopg2.pool import ThreadedConnectionPool

import pandas as pd
from tqdm import tqdm
import json

In [2]:
#set env
## env file for gcs upload
env_path = ".env"
with open(env_path) as f:
    env = {}
    for line in f:
        env_key, _val = line.split("=", 1)
        env_value = _val.split("\n")[0]
        env[env_key] = env_value
        
#list(env.keys())

# set conexion to local ddbb
postgres_thread_pool = ThreadedConnectionPool(1, 50,
                                              host=env['API_POSTGRES_HOST'],
                                              port=env['API_POSTGRES_PORT'],
                                              user=env['API_POSTGRES_USERNAME'],
                                              password=env['API_POSTGRES_PASSWORD']
                                              )

#get list of sourcing records to iterate:
conn = postgres_thread_pool.getconn()
cursor = conn.cursor()

## Option 1 - calculation of impact with probability purchase area:

In [3]:

## query with ha - commodity

#select sum(risk.wrisk * locations.ppa) Totalimpact from
#	(select h3_uncompact(gr."h3Compact"::h3index[],6) h3index , sr.tonnage * (hatable."spam2010V2R0GlobalHCottA" / sum(hatable."spam2010V2R0GlobalHCottA") over()) as ppa from sourcing_records sr 
#	left join sourcing_location sl on sl.id = sr."sourcingLocationId" 
#	left join geo_region gr on sl."geoRegionId" =gr.id 
#	left join h3_grid_spam2010v2r0_global_ha hatable on hatable.h3index = h3index
#	where sr.id='45cf7358-2a9a-44cc-9931-6f635f1fae93' and hatable."spam2010V2R0GlobalHCottA" > 0) locations
#left join
#	(select ind.h3index, (ind."wfBltotMmyr"* 0.001) / materials.totalProd wrisk from h3_grid_wf_global ind
#	left join 
#		(select spam_prod.h3index, sum(spam_prod."spam2010V2R0GlobalPPotaA" + spam_prod."spam2010V2R0GlobalPRapeA" + spam_prod."spam2010V2R0GlobalPCocoA" + spam_prod."spam2010V2R0GlobalPBarlA" + spam_prod."spam2010V2R0GlobalPOilpA" + spam_prod."spam2010V2R0GlobalPSunfA" + spam_prod."spam2010V2R0GlobalPSorgA" + spam_prod."spam2010V2R0GlobalPMaizA" + spam_prod."spam2010V2R0GlobalPRiceA" + spam_prod."spam2010V2R0GlobalPSugbA" + spam_prod."spam2010V2R0GlobalPBanaA" + spam_prod."spam2010V2R0GlobalPWheaA" + spam_prod."spam2010V2R0GlobalPTobaA" + spam_prod."spam2010V2R0GlobalPSwpoA" + spam_prod."spam2010V2R0GlobalPSoybA" + spam_prod."spam2010V2R0GlobalPOcerA" + spam_prod."spam2010V2R0GlobalPSugcA" + spam_prod."spam2010V2R0GlobalPCottA" + spam_prod."spam2010V2R0GlobalPTeasA" + spam_prod."spam2010V2R0GlobalPOoilA") as totalProd
#		from h3_grid_spam2010v2r0_global_prod spam_prod
#		group by spam_prod.h3index) materials
#	on ind.h3index=materials.h3index
#	where ind."wfBltotMmyr" > 0 and materials.totalProd > 0) risk
#on risk.h3index=locations.h3index
#where risk.wrisk > 0 and locations.ppa >0


## Option 2 - calculation of impact with average in location * Volume:

In [4]:
def get_indicatorRecordVal(sourcingRecord):
    #get material id for sourcing record
    sql_materialId= f"""select sl."materialId"  from sourcing_records sr 
    left join sourcing_location sl on sl.id =Sr."sourcingLocationId" 
    where sr.id ='{sourcingRecord}'"""
    cursor.execute(sql_materialId)
    response_materialId = cursor.fetchall()


    #get production tables for materials
    sql_prod_tables = f"""select hd."h3tableName", hd."h3columnName" from h3_data hd where hd.id in (
    select mth."h3DataId" from material_to_h3 mth where mth."materialId" = '{response_materialId[0][0]}' and mth."type" ='producer')"""
    cursor.execute(sql_prod_tables)
    response_prodtables = cursor.fetchall()
    prod_table= response_prodtables[0][0]
    prod_column =  response_prodtables[0][1]

    ## get harvest tables 
    sql_ha_tables = f"""select hd."h3tableName", hd."h3columnName" from h3_data hd where hd.id in (
    select mth."h3DataId" from material_to_h3 mth where mth."materialId" = '{response_materialId[0][0]}' and mth."type" ='producer')"""
    cursor.execute(sql_ha_tables)
    response_hatables = cursor.fetchall()
    ha_table= response_hatables[0][0]
    ha_column =  response_hatables[0][1]


    ## water indicator factor ####
    # Water ipact  (m3 / year) = volume (tonnes) * average(water footprint (m3/year)/ Total Production (tons)) 
    # note: the prod values are hardcoded for all water calculations
    sql_wi = f"""select avg(risk.wrisk)*locations.tonnage as wimpact from
        (select h3_uncompact(gr."h3Compact"::h3index[],6) h3index , sr.tonnage from sourcing_records sr 
        left join sourcing_location sl on sl.id = sr."sourcingLocationId" 
        left join geo_region gr on sl."geoRegionId" =gr.id 
        where sr.id='{sourcingRecord}') locations
    left join 
        (select ind.h3index, (ind."wfBltotMmyr"* 0.001) / materials.totalProd wrisk from h3_grid_wf_global ind
    join 
        (select spam_prod.h3index, sum(spam_prod."spam2010V2R0GlobalPPotaA" + spam_prod."spam2010V2R0GlobalPRapeA" + spam_prod."spam2010V2R0GlobalPCocoA" + spam_prod."spam2010V2R0GlobalPBarlA" + spam_prod."spam2010V2R0GlobalPOilpA" + spam_prod."spam2010V2R0GlobalPSunfA" + spam_prod."spam2010V2R0GlobalPSorgA" + spam_prod."spam2010V2R0GlobalPMaizA" + spam_prod."spam2010V2R0GlobalPRiceA" + spam_prod."spam2010V2R0GlobalPSugbA" + spam_prod."spam2010V2R0GlobalPBanaA" + spam_prod."spam2010V2R0GlobalPWheaA" + spam_prod."spam2010V2R0GlobalPTobaA" + spam_prod."spam2010V2R0GlobalPSwpoA" + spam_prod."spam2010V2R0GlobalPSoybA" + spam_prod."spam2010V2R0GlobalPOcerA" + spam_prod."spam2010V2R0GlobalPSugcA" + spam_prod."spam2010V2R0GlobalPCottA" + spam_prod."spam2010V2R0GlobalPTeasA" + spam_prod."spam2010V2R0GlobalPOoilA") as totalProd
        from h3_grid_spam2010v2r0_global_prod spam_prod
        group by spam_prod.h3index) materials
    on ind.h3index=materials.h3index
    where ind."wfBltotMmyr" > 0 and materials.totalProd > 0) risk
    on risk.h3index= locations.h3index
    group by locations.tonnage"""
    cursor.execute(sql_wi)
    response_wi_ir = cursor.fetchall()
    #print(response_wi_ir[0][0])


     ## deforestation indicator factor  ####
    # Deforestation impact (ha/ year) = volume (tonnes) + average(deforestation risk (unitless) * Harvested area (ha)/ Total Production (tons))
    sql_di = f"""select locations.tonnage * avg((materials.ha/materials.prod) * materials.defmask) as landimpact	from
        (select h3_uncompact(gr."h3Compact"::h3index[],6) h3index , sr.tonnage from sourcing_records sr 
        left join sourcing_location sl on sl.id = sr."sourcingLocationId" 
        left join geo_region gr on sl."geoRegionId" =gr.id 
        where sr.id='{sourcingRecord}') locations
    inner join 	
        (select prodTable.h3index, prodTable."{prod_column}" prod , haTable."{ha_column}"  ha, def."hansenLoss2019" defmask from {prod_table} prodTable
        left join {ha_table} haTable on haTable.h3index =prodTable.h3index 
        left join h3_grid_deforestation_global def on def.h3index = prodTable.h3index --indicator deforestation
        where prodTable."{prod_column}">0 and haTable."{ha_column}"> 0 and def."hansenLoss2019" > 0) materials
    on locations.h3index=materials.h3index
    group by locations.tonnage"""
    cursor.execute(sql_di)
    response_di_ir = cursor.fetchall()
    #print(response_di_ir[0][0])

    ## carbon indicator factor ####
    # Carbon impact  (tCO2eq / year ) = volume (tonnes) * average(carbon risk (tCO2eq / ha) * Deforestation risk (ha/tons) )
    sql_ci = f"""select locations.tonnage * avg((materials.ha/materials.prod) * materials.defmask * materials.carbon) as carbonimpact	from
        (select h3_uncompact(gr."h3Compact"::h3index[],6) h3index , sr.tonnage from sourcing_records sr 
        left join sourcing_location sl on sl.id = sr."sourcingLocationId" 
        left join geo_region gr on sl."geoRegionId" =gr.id 
        where sr.id='{sourcingRecord}') locations
    inner join 	
        (select prodTable.h3index, prodTable."{prod_column}" prod , haTable."{ha_column}"  ha, def."hansenLoss2019" defmask, ind."earthstat2000GlobalHectareEmissions" carbon from {prod_table} prodTable
        left join {ha_table} haTable on haTable.h3index =prodTable.h3index 
        left join h3_grid_deforestation_global def on def.h3index = prodTable.h3index --hardcoded deforestation h3 dataset
        left join h3_grid_carbon_global ind on ind.h3index = prodTable.h3index --carbon dataset
        where prodTable."{prod_column}">0 and haTable."{ha_column}"> 0 and def."hansenLoss2019" > 0 and ind."earthstat2000GlobalHectareEmissions" is not null) materials
    on locations.h3index=materials.h3index
    group by locations.tonnage"""
    cursor.execute(sql_ci)
    response_ci_ir = cursor.fetchall()
    #print(response_ci_ir[0][0])

    ## biodiversity indicator factor ####
    ## biodiversity impact  (PDF / year) =  volume(tonnes) * average(biodiversity risk ( PDF/year *m2)*(1/0.0001) unit conversion )
    sql_bi = f"""select locations.tonnage * avg((materials.ha/materials.prod) * materials.defmask * materials.bio * (1/0.0001)) as bioimpact	from
        (select h3_uncompact(gr."h3Compact"::h3index[],6) h3index , sr.tonnage from sourcing_records sr 
        left join sourcing_location sl on sl.id = sr."sourcingLocationId" 
        left join geo_region gr on sl."geoRegionId" =gr.id 
        where sr.id='{sourcingRecord}') locations
    inner join 	
        (select prodTable.h3index, prodTable."{prod_column}" prod , haTable."{ha_column}"  ha, def."hansenLoss2019" defmask, ind."lciaPslRPermanentCrops" bio from {prod_table} prodTable
        left join {ha_table} haTable on haTable.h3index =prodTable.h3index 
        left join h3_grid_deforestation_global def on def.h3index = prodTable.h3index  --hardcoded deforestation h3 dataset
        left join h3_grid_bio_global ind on ind.h3index = prodTable.h3index -- indicator dataset
        where prodTable."{prod_column}">0 and haTable."{ha_column}"> 0 and def."hansenLoss2019" > 0 and ind."lciaPslRPermanentCrops" is not null) materials
    on locations.h3index=materials.h3index
    group by locations.tonnage"""
    cursor.execute(sql_bi)
    response_bi_ir = cursor.fetchall()
    #print(response_bi_ir[0][0])
    
    # Land impact (ha)  = avg((Harvested area (ha) / Production)) * Volume
    sql_li = f"""select locations.tonnage * avg(materials.ha/materials.prod) as landrisk	from
        (select h3_uncompact(gr."h3Compact"::h3index[],6) h3index , sr.tonnage from sourcing_records sr 
        left join sourcing_location sl on sl.id = sr."sourcingLocationId" 
        left join geo_region gr on sl."geoRegionId" =gr.id 
        where sr.id='{sourcingRecord}') locations
    inner join 	
        (select prodTable.h3index, prodTable."{prod_column}" prod , haTable."{ha_column}"  ha from {prod_table} prodTable
        left join {ha_table} haTable on haTable.h3index =prodTable.h3index 
        where prodTable."{prod_column}">0 and haTable."{ha_column}"> 0) materials
    on locations.h3index=materials.h3index
    group by locations.tonnage"""
    cursor.execute(sql_li)
    response_li_ir = cursor.fetchall()
    
    return {
        'carbon_emissions_tCO2e': response_ci_ir[0][0],#carbon_emissions_tCO2e_t #c71eb531-2c8e-40d2-ae49-1049543be4d1
        'land_ha': response_li_ir[0][0],
        'deforestation_ha':response_di_ir[0][0], #deforestation_ha #633cf928-7c4f-41a3-99c5-e8c1bda0b323
        'biodiversity_impact_PDF': response_bi_ir[0][0], #biodiversity_impact_PDF #0594aba7-70a5-460c-9b58-fc1802d264ea
        'water_use_m3':response_wi_ir[0][0] #water_use_m3 #e2c00251-fe31-4330-8c38-604535d795dc
        
    }

In [5]:
#get indicators records from ddbb
#get list of sourcing records to iterate:
conn = postgres_thread_pool.getconn()
cursor = conn.cursor()

sql_sr = """select sr.id  from sourcing_records sr """
cursor.execute(sql_sr)
response_srIds = cursor.fetchall()

indicator_record = []
for sr in tqdm(response_srIds):
    try:
        res = get_indicatorRecordVal(sr[0])
        indicator_record.append({sr[0]:res})
    except:
         indicator_record.append({sr[0]:{
        'carbon_emissions_tCO2e': None,#carbon_emissions_tCO2e
        'land_ha': None, #land_ha
        'deforestation_ha': None, #deforestation_ha
        'biodiversity_impact_PDF': None, #biodiversity_impact_PDF
        'water_use_m3': None #water_use_m3      
    }})

100%|██████████| 495/495 [33:08<00:00,  4.02s/it]


In [6]:
indicator_record

[{'4057312f-131b-44a3-86cb-4ff6253026ce': {'carbon_emissions_tCO2e': 1639.7071394575646,
   'land_ha': 650.0,
   'deforestation_ha': 650.0,
   'biodiversity_impact_PDF': 102995186.41219082,
   'water_use_m3': 1.436334557207781}},
 {'2280336e-f744-4ab0-a1e6-9c3a9e83c62b': {'carbon_emissions_tCO2e': 1657.3655240363385,
   'land_ha': 657.0,
   'deforestation_ha': 657.0,
   'biodiversity_impact_PDF': 104104365.34278364,
   'water_use_m3': 1.4518027755161726}},
 {'1926c1dc-caf5-467b-93da-b958ca512865': {'carbon_emissions_tCO2e': 1675.0239086151123,
   'land_ha': 664.0,
   'deforestation_ha': 664.0,
   'biodiversity_impact_PDF': 105213544.27337648,
   'water_use_m3': 1.467270993824564}},
 {'c8266f82-5b40-4ffb-a6c8-42cf445064f4': {'carbon_emissions_tCO2e': 1692.682293193886,
   'land_ha': 671.0,
   'deforestation_ha': 671.0,
   'biodiversity_impact_PDF': 106322723.2039693,
   'water_use_m3': 1.4827392121329557}},
 {'b6488db3-165a-4123-9313-abc91da76381': {'carbon_emissions_tCO2e': 1710.340677

In [7]:
# get water outputs
irs= []
water_ir = []
def_ir = []
carb_ir = []
bio_ir= []
for el in indicator_record:
    for sr in el:
        irs.append(sr)
        try:
            water_ir.append(el[sr]['water_use_m3'])
            def_ir.append(el[sr]['deforestation_ha'])
            carb_ir.append(el[sr]['carbon_emissions_tCO2e'])
            bio_ir.append(el[sr]['biodiversity_impact_PDF'])
        except:
            water_ir.append(None)
            def_ir.append(None)
            carb_ir.append(None)            
            bio_ir.append(None)

In [8]:
df_ = pd.DataFrame(irs, columns = {'IndicatorRecord_id'})
df_['es_water_use_m3']=water_ir
df_['es_deforestation_ha']=def_ir
df_['es_carbon_emissions_tCO2e']=carb_ir
df_['es-biodiversity_impact_PDF']=bio_ir
df_.head()

,IndicatorRecord_id,es_water_use_m3,es_deforestation_ha,es_carbon_emissions_tCO2e,es-biodiversity_impact_PDF
0,4057312f-131b-44a3-86cb-4ff6253026ce,1.436335,650.0,1639.707139,1.029952e+08
1,2280336e-f744-4ab0-a1e6-9c3a9e83c62b,1.451803,657.0,1657.365524,1.041044e+08
2,1926c1dc-caf5-467b-93da-b958ca512865,1.467271,664.0,1675.023909,1.052135e+08
3,c8266f82-5b40-4ffb-a6c8-42cf445064f4,1.482739,671.0,1692.682293,1.063227e+08
4,b6488db3-165a-4123-9313-abc91da76381,1.498207,678.0,1710.340678,1.074319e+08


In [9]:
len(df_)

495

In [11]:
#retrive information that we have right now on the ddbb
sql_ddbb_ir = """select sr.id, i."shortName", ir.value from sourcing_records sr 
right join indicator_record ir on ir."sourcingRecordId" =sr.id
left join "indicator" i on i.id = ir."indicatorId" """
cursor.execute(sql_ddbb_ir)
response_ir_ddbb = cursor.fetchall()


In [40]:
#clean response 
water_db_list = [el for el in response_ir_ddbb if el[1]=='Unsustainable water use']
def_db_list = [el for el in response_ir_ddbb if el[1]=='Deforestation loss']
carb_db_list = [el for el in response_ir_ddbb if el[1]=='GHG emissions']
bio_db_list = [el for el in response_ir_ddbb if el[1]=='biodiversity loss']


db_water_val = []
db_def_val = []
db_carb_val = []
db_bio_val = []

for ir in irs:
    
    w_val = [el[2] for el in water_db_list if el[0]==ir]
    d_val = [el[2] for el in def_db_list if el[0]==ir]
    c_val = [el[2] for el in carb_db_list if el[0]==ir]
    b_val = [el[2] for el in bio_db_list if el[0]==ir]
    
    #get water val
    if len(w_val)==0:
        db_water_val.append(0)
    else:
        db_water_val.append(w_val[0])
        
    #get def val
    if len(w_val)==0:
        db_def_val.append(0)
    else:
        db_def_val.append(d_val[0])

    #get carb val
    if len(c_val)==0:
        db_carb_val.append(0)
    else:
        db_carb_val.append(c_val[0])

    #get bio val
    if len(b_val)==0:
        db_bio_val.append(0)
    else:
        db_bio_val.append(b_val[0])

In [42]:
#append to dataframe
df_['db_water_use_m3']=db_water_val
df_['db_deforestation_ha']=db_def_val
df_['db_carbon_emissions_tCO2e']=db_carb_val
df_['db_biodiversity_impact_PDF']=db_bio_val
df_.head()

,IndicatorRecord_id,es_water_use_m3,es_deforestation_ha,es_carbon_emissions_tCO2e,es-biodiversity_impact_PDF,db_water_use_m3,db_deforestation_ha,db_carbon_emissions_tCO2e,db_biodiversity_impact_PDF
0,4057312f-131b-44a3-86cb-4ff6253026ce,1.436335,650.0,1639.707139,1.029952e+08,0.0,0.0,0.0,0.0
1,2280336e-f744-4ab0-a1e6-9c3a9e83c62b,1.451803,657.0,1657.365524,1.041044e+08,0.0,0.0,0.0,0.0
2,1926c1dc-caf5-467b-93da-b958ca512865,1.467271,664.0,1675.023909,1.052135e+08,0.0,0.0,0.0,0.0
3,c8266f82-5b40-4ffb-a6c8-42cf445064f4,1.482739,671.0,1692.682293,1.063227e+08,0.0,0.0,0.0,0.0
4,b6488db3-165a-4123-9313-abc91da76381,1.498207,678.0,1710.340678,1.074319e+08,0.0,0.0,0.0,0.0


In [51]:
#order df to compute differences with previous column
df_2 = df_[['es_water_use_m3','db_water_use_m3', 'es_deforestation_ha', 'db_deforestation_ha', 'es_carbon_emissions_tCO2e', 'db_carbon_emissions_tCO2e','es-biodiversity_impact_PDF', 'db_biodiversity_impact_PDF']]
df_diff =  df_2.diff(axis=1)
df_diff

,es_water_use_m3,db_water_use_m3,es_deforestation_ha,db_deforestation_ha,es_carbon_emissions_tCO2e,db_carbon_emissions_tCO2e,es-biodiversity_impact_PDF,db_biodiversity_impact_PDF
0,NaN,-1.436335,650.0,-650.0,1639.707139,-1639.707139,1.029952e+08,-1.029952e+08
1,NaN,-1.451803,657.0,-657.0,1657.365524,-1657.365524,1.041044e+08,-1.041044e+08
2,NaN,-1.467271,664.0,-664.0,1675.023909,-1675.023909,1.052135e+08,-1.052135e+08
3,NaN,-1.482739,671.0,-671.0,1692.682293,-1692.682293,1.063227e+08,-1.063227e+08
4,NaN,-1.498207,678.0,-678.0,1710.340678,-1710.340678,1.074319e+08,-1.074319e+08
...,...,...,...,...,...,...,...,...
490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
492,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
df_diff

,es_water_use_m3,db_water_use_m3,es_deforestation_ha,db_deforestation_ha,es_carbon_emissions_tCO2e,db_carbon_emissions_tCO2e,es-biodiversity_impact_PDF,db_biodiversity_impact_PDF
0,NaN,-1.436335,650.0,-650.0,1639.707139,-1639.707139,1.029952e+08,-1.029952e+08
1,NaN,-1.451803,657.0,-657.0,1657.365524,-1657.365524,1.041044e+08,-1.041044e+08
2,NaN,-1.467271,664.0,-664.0,1675.023909,-1675.023909,1.052135e+08,-1.052135e+08
3,NaN,-1.482739,671.0,-671.0,1692.682293,-1692.682293,1.063227e+08,-1.063227e+08
4,NaN,-1.498207,678.0,-678.0,1710.340678,-1710.340678,1.074319e+08,-1.074319e+08
...,...,...,...,...,...,...,...,...
490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
492,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
